In [1]:
%reload_ext autoreload
%autoreload 2

The overlap sheet inventarizes the incidence and overlap between the different 'communities' we discern in the data. We will try to derive the importance and the hubs of the networks from them. Even if they actually measure different things (either number of publications in a journal or positions in an institution), we can still measure their presence in all the networks. Therefore we have a measure that sums the combined weight of a person (the sum of all columns), but more importantly the count of the number of networks the person is present in. 

We first load the spreadsheet from google sheets and convert them to a pandas dataframe. As they are transferred over the web, we need to convert the columns to numeric values.

In [187]:
overlap_records = retrieve_spreadsheet_records(record_type='overlap')
print('Number of records:' , len(overlap_records))

Number of records: 2498


In [188]:
overlap_df = pd.DataFrame().from_records(overlap_records)

In [189]:
for c in overlap_df.columns:
    try:
        overlap_df[c] = pd.to_numeric(overlap_df[c], errors='ignore')
    except (ValueError, KeyError):
        pass

In [190]:
overlap_df.describe()

,nr,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,Overlap_IMR_IM
count,2498.000000,2498.000000,2498.000000,2498.000000,2498.000000,2498.000000,2498.000000,2498.000000
mean,1248.500000,1.340272,0.062050,0.352282,0.001201,0.003203,0.008407,0.045637
std,721.254809,1.792474,0.382529,0.861212,0.034641,0.056512,0.091320,0.208737
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,624.250000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1248.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1872.750000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,2497.000000,25.000000,12.000000,19.000000,1.000000,1.000000,1.000000,1.000000


In [191]:
overlap_df.columns

Index(['nr', 'Author', 'IMR-Sage', 'REMP-Staatsdrukkerij', 'IM-Wiley',
       'Dutch_Government', 'ICEM', 'REMP', 'Overlap_IMR_IM'],
      dtype='object')

In [220]:
numcols= ['IMR-Sage', 'REMP-Staatsdrukkerij', 'IM-Wiley',
       'Dutch_Government', 'ICEM', 'REMP', 'Overlap_IMR_IM']


First we make the sum of values

In [192]:
overlap_df['total'] = overlap_df[['IMR-Sage', 'REMP-Staatsdrukkerij', 'IM-Wiley',
       'Dutch_Government', 'ICEM', 'REMP', 'Overlap_IMR_IM']].sum(axis=1)

In [193]:
overlap_df.sort_values(['total'], ascending=False)

,nr,Author,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,Overlap_IMR_IM,total
207,207,"Beyer, G",3.0,12.0,9.0,0.0,0.0,1.0,1,26.0
2238,2238,"Tomasi, S",25.0,0.0,0.0,0.0,0.0,0.0,0,25.0
82,82,"Appleyard, R",1.0,2.0,19.0,0.0,0.0,1.0,1,24.0
287,287,"Brown, M",20.0,0.0,0.0,0.0,0.0,0.0,0,20.0
1450,1450,"Martin, P",17.0,0.0,1.0,0.0,0.0,0.0,1,19.0
...,...,...,...,...,...,...,...,...,...,...
981,981,"Hollifield, J",1.0,0.0,0.0,0.0,0.0,0.0,0,1.0
980,980,"Holborn, L",0.0,0.0,1.0,0.0,0.0,0.0,0,1.0
978,978,"Hogan, J",1.0,0.0,0.0,0.0,0.0,0.0,0,1.0
975,975,"Hofmeye, D",0.0,0.0,1.0,0.0,0.0,0.0,0,1.0


and then the count

In [230]:
def countcols(x):
    return len([c for c in x[2:8] if c>0])

In [231]:
from statistics import mean
def meancols(x):
    return mean([c for c in x[2:8]])

In [232]:
overlap_df["network_count"] = overlap_df.apply(lambda x: countcols(x), axis=1)

In [233]:
overlap_df["network_mean"] = overlap_df.apply(lambda x: meancols(x), axis=1)

In [234]:
overlap_df.sort_values(['network_count','network_mean','total'], ascending=False)

,nr,Author,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,Overlap_IMR_IM,total,network_count,network_mean
207,207,"Beyer, G",3.0,12.0,9.0,0.0,0.0,1.0,1,26.0,4,4.166667
82,82,"Appleyard, R",1.0,2.0,19.0,0.0,0.0,1.0,1,24.0,4,3.833333
190,190,"Bernard, W",6.0,1.0,6.0,0.0,0.0,0.0,1,14.0,3,2.166667
2495,2495,"Zubrzycki, J",3.0,3.0,6.0,0.0,0.0,0.0,1,13.0,3,2.000000
1573,1573,"Moncarz, R",1.0,1.0,8.0,0.0,0.0,0.0,1,11.0,3,1.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
2490,2490,"Zizek, F",1.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1,0.166667
2491,2491,"Zlotmk, H",1.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1,0.166667
2493,2493,"Zodgekar, A",0.0,0.0,1.0,0.0,0.0,0.0,0,1.0,1,0.166667
2496,2496,"Zucchi, J",1.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1,0.166667


In [235]:
counted_values = overlap_df.network_count.value_counts()
vals = list(counted_values.index)
vals.sort(reverse=True)

In [242]:
counted_values

1    2343
2     139
3      14
4       2
Name: network_count, dtype: int64

In [236]:
vals

[4, 3, 2, 1]

In [237]:
mean_values = overlap_df.network_mean.value_counts()
means = list(mean_values.index)
means.sort(reverse=True)


In [243]:
mean_values

0.166667    1742
0.333333     386
0.500000     160
0.666667      64
0.833333      49
1.000000      30
1.333333      16
1.166667      10
1.666667       9
1.833333       6
2.000000       5
2.333333       4
2.166667       4
2.500000       3
1.500000       3
4.166667       2
3.000000       2
2.833333       1
3.833333       1
3.333333       1
Name: network_mean, dtype: int64

In [238]:
means

[4.166666666666667,
 3.8333333333333335,
 3.3333333333333335,
 3.0,
 2.8333333333333335,
 2.5,
 2.3333333333333335,
 2.1666666666666665,
 2.0,
 1.8333333333333333,
 1.6666666666666667,
 1.5,
 1.3333333333333333,
 1.1666666666666667,
 1.0,
 0.8333333333333334,
 0.6666666666666666,
 0.5,
 0.3333333333333333,
 0.16666666666666666]

which means that there are 2 persons who appear in 6 networks and 2343 who appear in 2 networks. Who are the ones who appear in more networks?

In [239]:
#this is for inline rendering an HTML table
def HTML_with_style(df, style=None, random_id=None):
    from IPython.display import HTML
    import numpy as np
    import re

    df_html = df.to_html()

    if random_id is None:
        random_id = 'id%d' % np.random.choice(np.arange(1000000))

    if style is None:
        style = """
        <style>
            table#{random_id} {{color: black}}
        </style>
        """.format(random_id=random_id)
    else:
        new_style = []
        s = re.sub(r'</?style>', '', style).strip()
        for line in s.split('\n'):
                line = line.strip()
                if not re.match(r'^table', line):
                    line = re.sub(r'^', 'table ', line)
                new_style.append(line)
        new_style = ['<style>'] + new_style + ['</style>']

        style = re.sub(r'table(#\S+)?', 'table#%s' % random_id, '\n'.join(new_style))

    df_html = re.sub(r'<table', r'<table id=%s ' % random_id, df_html)

    return HTML(style + df_html)


In [241]:
from IPython.core.display import display, HTML
for i in vals:
    if i>1: # we only want those who appear frequently
        display(HTML(f'{i} networks'))
        res = overlap_df.loc[overlap_df.network_count==i]
        display(HTML_with_style(res))

,nr,Author,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,Overlap_IMR_IM,total,network_count,network_mean
82,82,"Appleyard, R",1.0,2.0,19.0,0.0,0.0,1.0,1,24.0,4,3.833333
207,207,"Beyer, G",3.0,12.0,9.0,0.0,0.0,1.0,1,26.0,4,4.166667


,nr,Author,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,Overlap_IMR_IM,total,network_count,network_mean
103,103,"Avila, F",0.0,2.0,1.0,0.0,0.0,1.0,0,4.0,3,0.666667
190,190,"Bernard, W",6.0,1.0,6.0,0.0,0.0,0.0,1,14.0,3,2.166667
250,250,"Borrie, W",0.0,1.0,1.0,0.0,0.0,1.0,0,3.0,3,0.500000
918,918,"Haveman, B",0.0,0.0,4.0,1.0,1.0,0.0,0,6.0,3,1.000000
1459,1459,"Maselli, G",0.0,1.0,2.0,0.0,1.0,0.0,0,4.0,3,0.666667
1573,1573,"Moncarz, R",1.0,1.0,8.0,0.0,0.0,0.0,1,11.0,3,1.666667
1821,1821,"Price, C",7.0,1.0,2.0,0.0,0.0,0.0,1,11.0,3,1.666667
1877,1877,"Richardson, A",3.0,3.0,2.0,0.0,0.0,0.0,1,9.0,3,1.333333
1997,1997,"Sauvy, A",0.0,1.0,1.0,0.0,0.0,1.0,0,3.0,3,0.500000
2192,2192,"Taft, R",2.0,1.0,1.0,0.0,0.0,0.0,1,5.0,3,0.666667


,nr,Author,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,Overlap_IMR_IM,total,network_count,network_mean
24,24,"Adepoju, A",1.0,0.0,5.0,0.0,0.0,0.0,1,7.0,2,1.000000
30,30,"Agarwal, V",1.0,0.0,1.0,0.0,0.0,0.0,1,3.0,2,0.333333
50,50,"Alexander, R",1.0,1.0,0.0,0.0,0.0,0.0,0,2.0,2,0.333333
88,88,"Arnold, F",5.0,0.0,1.0,0.0,0.0,0.0,1,7.0,2,1.000000
96,96,"Athukorala, P",1.0,0.0,2.0,0.0,0.0,0.0,1,4.0,2,0.500000
111,111,"Backer, J",0.0,0.0,1.0,0.0,0.0,1.0,0,2.0,2,0.333333
156,156,"Baskauskas, L",1.0,0.0,1.0,0.0,0.0,0.0,1,3.0,2,0.333333
169,169,"Beenstock, M",1.0,0.0,2.0,0.0,0.0,0.0,1,4.0,2,0.500000
194,194,"Berry, J",1.0,0.0,1.0,0.0,0.0,0.0,1,3.0,2,0.333333
203,203,"Besterman, W",0.0,0.0,2.0,0.0,1.0,0.0,0,3.0,2,0.500000


In [219]:
from IPython.core.display import display, HTML
style = "style='display:inline'"


for i in vals:
    if i>2: # we only want those who appear frequently
        res = overlap_df.loc[overlap_df.network_count==i]
        #res = res[numcols].apply(lambda x: x.round(0))
        display(res.style.set_table_attributes(style).set_caption(f'{i} networks'))

,nr,Author,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,Overlap_IMR_IM,total,network_count,network_mean
82,82,"Appleyard, R",1.000000,2.000000,19.000000,0.000000,0.000000,1.000000,1,24.000000,6,6.000000
207,207,"Beyer, G",3.000000,12.000000,9.000000,0.000000,0.000000,1.000000,1,26.000000,6,6.444444


,nr,Author,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,Overlap_IMR_IM,total,network_count,network_mean
190,190,"Bernard, W",6.000000,1.000000,6.000000,0.000000,0.000000,0.000000,1,14.000000,5,3.666667
1573,1573,"Moncarz, R",1.000000,1.000000,8.000000,0.000000,0.000000,0.000000,1,11.000000,5,3.000000
1821,1821,"Price, C",7.000000,1.000000,2.000000,0.000000,0.000000,0.000000,1,11.000000,5,3.000000
1877,1877,"Richardson, A",3.000000,3.000000,2.000000,0.000000,0.000000,0.000000,1,9.000000,5,2.555556
2192,2192,"Taft, R",2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1,5.000000,5,1.666667
2214,2214,"Thomas, B",2.000000,0.000000,2.000000,0.000000,0.000000,1.000000,1,6.000000,5,1.888889
2215,2215,"Thomas, J",1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1,4.000000,5,1.444444
2495,2495,"Zubrzycki, J",3.000000,3.000000,6.000000,0.000000,0.000000,0.000000,1,13.000000,5,3.444444


,nr,Author,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,Overlap_IMR_IM,total,network_count,network_mean
24,24,"Adepoju, A",1.000000,0.000000,5.000000,0.000000,0.000000,0.000000,1,7.000000,4,2.000000
30,30,"Agarwal, V",1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1,3.000000,4,1.111111
88,88,"Arnold, F",5.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1,7.000000,4,2.000000
96,96,"Athukorala, P",1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1,4.000000,4,1.333333
103,103,"Avila, F",0.000000,2.000000,1.000000,0.000000,0.000000,1.000000,0,4.000000,4,1.333333
156,156,"Baskauskas, L",1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1,3.000000,4,1.111111
169,169,"Beenstock, M",1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1,4.000000,4,1.333333
194,194,"Berry, J",1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1,3.000000,4,1.111111
215,215,"Birks, J",3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1,5.000000,4,1.555556
238,238,"Bohning, W",6.000000,0.000000,4.000000,0.000000,0.000000,0.000000,1,11.000000,4,2.888889


,nr,Author,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,Overlap_IMR_IM,total,network_count,network_mean
50,50,"Alexander, R",1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0,2.000000,3,0.777778
111,111,"Backer, J",0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0,2.000000,3,0.777778
203,203,"Besterman, W",0.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0,3.000000,3,1.000000
256,256,"Bouscaren, A",6.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0,8.000000,3,2.111111
258,258,"Bovenkerk, F",2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0,3.000000,3,1.000000
277,277,"Brink, Van Den, T",0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0,3.000000,3,1.000000
329,329,"Carlin, J",0.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0,3.000000,3,1.000000
361,361,"Charry-Samper, H",0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0,2.000000,3,0.777778
403,403,"Cnossen, T",0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0,3.000000,3,1.000000
540,540,"Doczy, A",0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0,2.000000,3,0.777778
